<a href="https://colab.research.google.com/github/Thamarai-Selvam/Itunes-Podcast-DB-India/blob/master/Podcasts_data_miner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pattern3

import requests
from bs4 import BeautifulSoup
import pattern3.web as web
import re
import numpy as np
import pandas as pd 
from progressbar import ProgressBar
import matplotlib.pyplot as plt



site = 'https://podcasts.apple.com/in/genre/podcasts/id26'

def getlinks(site):
    try:
        url = url = web.URL(site)
        bs = BeautifulSoup(url.download(cached = False)) 
        links = []
        for link in bs.findAll('a', href=True):
            links.append(link['href'])
    except:
        print("Can't get any links bruh...") 
    return links

def genre_finder(links, pattern):
    genres = [] 
    for link in links:
            if re.search(pattern, link):
                genres.append(link)
    return genres


# fill table:

def getdata(ID):

    url = 'https://podcasts.apple.com/lookup?id=' + ID

    try:
        response = requests.get(url) 
        data = response.json()
        
        if data["resultCount"] != 0: 
            for elem in data["results"]:   
                if "kind" in elem.keys(): 
                    if elem["kind"] == "podcast":
                        if "trackCount" in elem.keys():
                            if elem["trackCount"] >= 20: 
                                if "artworkUrl100" in elem.keys():
                                    artwork = elem["artworkUrl100"]
                                if "feedUrl" in elem.keys(): 
                                    rss_feed = elem["feedUrl"]
                                if "genreIds" in elem.keys():
                                    genre_IDs = elem["genreIds"]
                                    
                                    
                                return [artwork, rss_feed, genre_IDs]
                            
                            else: 
                                return None 

                    else:
                        return None 
                          
        else: 
            return None

    except:
        None


def get_description(webpage):
    
    global df_podcast
    iTunes_URL = webpage
    podcast_id = re.findall('[0-9]+', re.findall('id[0-9]+', iTunes_URL)[0])[0]
    api_data = getdata(podcast_id)
    
    if api_data is not None: 
        artwork = api_data[0]
        feedurl = api_data[1]
        genre_IDs = api_data[2]
        
        
    
        try: 

            url = web.URL(webpage)
            bs = BeautifulSoup(url.download(cached = False)) 

            titles = bs.find('div', id='title')
            if titles is not None:
                title = titles.find('h1').getText()


            try:     
                podcast_left = bs.find('div', id='left-stack')
                if podcast_left is not None:
                    links = podcast_left.findAll('a', href=True)
                    if links is not None: 
                        for link in links:                   
                            if link.getText() == "Podcast Website":
                                podcast_link = link['href']

            except:
                podcast_link = 0



            try:              
                description = bs.find('div', 'product-review')
                if description is not None:
                    descr_parag = description.findAll('p')
                    for parag in descr_parag:
                        podcast_description = parag.getText()

            except:
                podcast_description = 0 




            try:        
                track_list = bs.find('div', 'track-list', 'podcast')
                episodes = track_list.findAll('tr', 'podcast-episode')
                episode_count = len(episodes)
                durations = [] 
                for episode in episodes:
                    if episode['preview-duration'] is not None:
                        duration = int(episode['preview-duration']) / 1000 / 60 
                        durations.append(float("{0:.2f}".format(duration)))
                    else:
                        duration = 0 
                        durations.append(duration)
    #             print (number_episodes, durations)    
            except: 
                durations = 0  


            my_row = [title, artwork, genre_IDs, episode_count, durations, iTunes_URL, feedurl, podcast_link, podcast_description]
            my_row_pd = pd.DataFrame([my_row], columns = columns)  
            df_podcast = df_podcast.append(my_row_pd, ignore_index=True)

            return df_podcast

        except: 
            return None 
        
    else: 
        return None





links = getlinks(site) # get links from the page
#print(*links, sep="\n")

gpattern = 'https://podcasts.apple.com/in/genre/podcasts-' # find genres
genres = genre_finder(links, gpattern)
#print('Genres and sub-genres found :', len(genres), *genres, sep="\n")

ppattern = 'https://podcasts.apple.com/in/podcast/'
podcast_links = []

for genre in genres:
    all_links = getlinks(genre)
    podcast_links.append(genre_finder(all_links, ppattern))

podcast_links2 = list(set(np.concatenate(podcast_links).tolist()))






columns= ['Name', 'Artwork', 'Genre IDs', 'Episode Count', 'Episode Durations','iTunes URL', 'Feed URL', 'Podcast URL', 'Description']

df_podcast = pd.DataFrame([['-', '-', '-', '-', '-', '-', '-', '-', '-']], columns = columns)
df_podcast


podcast_links3 = list(set(podcast_links2))
print (len(podcast_links2), len(podcast_links3))

pbar = ProgressBar()
for link in pbar(podcast_links2):
    get_description(link)

df_podcast.drop(df_podcast.index[0], inplace=True)
dfpodcast = df_podcast.reset_index(drop=True)
dfpodcast.to_csv("df_popular_podcasts.csv", index=False)

  0% (1 of 4861) |                       | Elapsed Time: 0:00:00 ETA:   0:13:50

4861 4861


100% (4861 of 4861) |####################| Elapsed Time: 0:13:35 Time:  0:13:35
